## Local alignment

#### Smith-Waterman

https://en.wikipedia.org/wiki/Smith–Waterman_algorithm
https://gtuckerkellogg.github.io/pairwise/demo/



In [9]:
import pandas as pd
import numpy as np

In [14]:
x='TGTTACGG'
y='GGTTGACTA'
indel = -2

In [11]:
def createScorMat(match = 1,mismatch = 0):
    """
    Create scoring matrix for nucleotides with a given match and mismatch values
    """
    nts = ['A','C','T','G']

    scoringMatrix = np.zeros((len(nts),len(nts)),)
    np.fill_diagonal(scoringMatrix,match)
    scoringMatrix
    scoringMatrix[scoringMatrix == 0] = mismatch
    scoringMatrix = pd.DataFrame(scoringMatrix,index=nts,columns=nts)
    return scoringMatrix

In [16]:
subMat = createScorMat(3,mismatch = -3)
subMat

,A,C,T,G
A,3.0,-3.0,-3.0,-3.0
C,-3.0,3.0,-3.0,-3.0
T,-3.0,-3.0,3.0,-3.0
G,-3.0,-3.0,-3.0,3.0
